In [1]:
# Gerekli kütüphaneleri ekleme

import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np
# import re
# from collections import defaultdict
# import geopandas
# import seaborn as sns

# sns.set()

In [2]:
# Dataseti okuma
data = pd.read_excel("Online Retail.xlsx")

In [3]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [5]:
# Bazı sütunların birbirine benzeri var mı diye kontrol için gereken fonksiyon

def isOneToMany(df, col1, col2):
    first = df.drop_duplicates([col1, col2]).groupby(col1)[col2].count().max()
    print(first == 1)

In [7]:
# Değeri Null olan CustomerID sütunlarını çıkarma
data = data.drop(data[data.CustomerID.isnull()].index).reset_index(drop=True)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406829 entries, 0 to 406828
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      406829 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 24.8+ MB


In [9]:
#  "InvoiceNo" Sütununu string tipini çevirme
data['InvoiceNo'] = data.InvoiceNo.astype('str')

In [10]:
#  "Description" sütununu da string'e çevirip fazla boşlukları silme
data['Description'] = data.Description.astype('str')
data['Description'] = data.Description.str.strip()

In [11]:
# Belli sütunların birbirileri arasında aynı değerleri olanları var mı diye kontrol etme

isOneToMany(data,'InvoiceNo','Country')
isOneToMany(data,'StockCode','Description')

True
False


In [12]:
# çift veri var mı diye kontrol

products = data.groupby(['StockCode','Description'],as_index=False).count()
products = products.loc[:,['StockCode','Description']]
stockCount = products.StockCode.value_counts().to_frame()
stockList = list(stockCount[stockCount.StockCode>1].index) #birden fazla description olanları listeleme
products[products.StockCode.isin(stockList)].head(10)

,StockCode,Description
48,20622,VIP PASSPORT COVER
49,20622,VIPPASSPORT COVER
101,20725,LUNCH BAG RED RETROSPOT
102,20725,LUNCH BAG RED SPOTTY
194,20914,SET/5 RED RETROSPOT LID GLASS BOWLS
195,20914,SET/5 RED SPOTTY LID GLASS BOWLS
290,21109,LARGE CAKE TOWEL CHOCOLATE SPOTS
291,21109,"LARGE CAKE TOWEL, CHOCOLATE SPOTS"
294,21112,SWISS ROLL TOWEL PINK SPOTS
295,21112,"SWISS ROLL TOWEL, PINK SPOTS"


In [13]:
# Aynı değerleri olanları çıkarma
products.drop_duplicates('StockCode', keep='first', inplace=True)

In [14]:
# StockCode'u Description'a doğru eşleme yapma 

df = pd.merge(data,products,on='StockCode', how='left',validate='many_to_one')
df.drop('Description_x',axis=1,inplace=True)
df.rename(columns={'Description_y':'Description'},inplace=True)
df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,CREAM HANGING HEART T-LIGHT HOLDER
1,536365,71053,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,WHITE METAL LANTERN
2,536365,84406B,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,CREAM CUPID HEARTS COAT HANGER
3,536365,84029G,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,KNITTED UNION FLAG HOT WATER BOTTLE
4,536365,84029E,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,RED WOOLLY HOTTIE WHITE HEART.


In [15]:
# Tekrar kontrol
isOneToMany(df,'StockCode','Description')

True


StockCode'u 5 digitten fazla olanların asıl işlem olması gerekiyor datasete bakıldığında

In [16]:
df[(df.StockCode.str.len())==6].sort_values(by='StockCode').head(10)

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description
96416,548491,10123C,1,2011-03-31 13:14:00,0.65,14064.0,United Kingdom,HEARTS WRAPPING TAPE
4165,536863,10123C,1,2010-12-03 11:19:00,0.65,17967.0,United Kingdom,HEARTS WRAPPING TAPE
72675,545655,10123C,3,2011-03-04 13:58:00,0.65,14670.0,United Kingdom,HEARTS WRAPPING TAPE
8987,537382,10124A,4,2010-12-06 13:13:00,0.42,16710.0,United Kingdom,SPOTS ON RED BOOKCOVER TAPE
53974,543397,10124A,3,2011-02-08 10:32:00,0.42,17859.0,United Kingdom,SPOTS ON RED BOOKCOVER TAPE
92138,547881,10124A,1,2011-03-27 16:08:00,0.42,13110.0,United Kingdom,SPOTS ON RED BOOKCOVER TAPE
83288,546913,10124A,4,2011-03-17 20:18:00,0.42,15861.0,United Kingdom,SPOTS ON RED BOOKCOVER TAPE
333228,574686,10124A,4,2011-11-06 13:00:00,0.42,17608.0,United Kingdom,SPOTS ON RED BOOKCOVER TAPE
83287,546913,10124G,4,2011-03-17 20:18:00,0.42,15861.0,United Kingdom,ARMY CAMO BOOKCOVER TAPE
270614,568938,10124G,4,2011-09-29 14:46:00,0.42,17220.0,United Kingdom,ARMY CAMO BOOKCOVER TAPE


In [17]:
df[(df.StockCode.str.len())<5].sort_values(by='StockCode').head(10)

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description
201605,561631,C2,1,2011-07-28 14:27:00,50.0,14911.0,EIRE,CARRIAGE
25993,539688,C2,1,2010-12-21 11:00:00,150.0,12678.0,France,CARRIAGE
155387,555858,C2,1,2011-06-07 14:12:00,50.0,14911.0,EIRE,CARRIAGE
66561,544934,C2,1,2011-02-25 09:01:00,50.0,13345.0,United Kingdom,CARRIAGE
389253,579910,C2,1,2011-12-01 08:52:00,50.0,14911.0,EIRE,CARRIAGE
26731,539984,C2,1,2010-12-23 14:58:00,50.0,14911.0,EIRE,CARRIAGE
179328,558908,C2,1,2011-07-05 09:06:00,50.0,13338.0,United Kingdom,CARRIAGE
242705,566261,C2,1,2011-09-11 12:46:00,50.0,13372.0,United Kingdom,CARRIAGE
315365,572969,C2,1,2011-10-27 10:46:00,50.0,13363.0,United Kingdom,CARRIAGE
27953,C540164,C2,-1,2011-01-05 12:02:00,50.0,14911.0,EIRE,CARRIAGE


5 digitten az olanları alt işlem olarak düşündüğümden o StockCode'a sahip işlemleri datasetten çıkarma

In [18]:
df = df.drop(df[(df.StockCode.str.len())<5].index).reset_index(drop=True)

In [19]:
df.describe(include='all')

<ipython-input-19-174ba9bf1a5c>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe(include='all')


,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description
count,404921,404921,404921.000000,404921,404921.000000,404921.000000,404921,404921
unique,21799,3677,NaN,20145,NaN,NaN,37,3652
top,576339,85123A,NaN,2011-11-14 15:27:00,NaN,NaN,United Kingdom,CREAM HANGING HEART T-LIGHT HOLDER
freq,541,2077,NaN,541,NaN,NaN,361267,2077
first,NaN,NaN,NaN,2010-12-01 08:26:00,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,2011-12-09 12:50:00,NaN,NaN,NaN,NaN
mean,NaN,NaN,12.105470,NaN,2.901451,15295.158048,NaN,NaN
std,NaN,NaN,249.233858,NaN,4.431232,1710.414161,NaN,NaN
min,NaN,NaN,-80995.000000,NaN,0.000000,12346.000000,NaN,NaN
25%,NaN,NaN,2.000000,NaN,1.250000,13969.000000,NaN,NaN


CustomerID'si C ile başlayanları float olarak gördüğü için onların düzeltimesi

In [20]:
df['CustomerID'] = 'C' + df.CustomerID.astype('int').astype('str')
df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,C17850,United Kingdom,CREAM HANGING HEART T-LIGHT HOLDER
1,536365,71053,6,2010-12-01 08:26:00,3.39,C17850,United Kingdom,WHITE METAL LANTERN
2,536365,84406B,8,2010-12-01 08:26:00,2.75,C17850,United Kingdom,CREAM CUPID HEARTS COAT HANGER
3,536365,84029G,6,2010-12-01 08:26:00,3.39,C17850,United Kingdom,KNITTED UNION FLAG HOT WATER BOTTLE
4,536365,84029E,6,2010-12-01 08:26:00,3.39,C17850,United Kingdom,RED WOOLLY HOTTIE WHITE HEART.


"InvoiceNo" sütununda C ile başlamayıp "Quantity"'si negatif olanlara baktık ki iptal olarak görünmeyip negatif değerde olanlar var mı diye kontrol edebilmek için

In [21]:
# Negatif satışlar için kontrol edildi

df[(df.Quantity<1) & (~df.InvoiceNo.str.startswith('C'))]

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description


Tüm negatif değerlerin iptal edilen siparişler olduğunu anladık

In [22]:
# 4000'den fazla olan Quantity değerlerine baktık
df[(abs(df.Quantity)>4000)]

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description
3095,C536757,84347,-9360,2010-12-02 14:23:00,0.03,C15838,United Kingdom,ROTATING SILVER ANGELS T-LIGHT HLDR
37950,541431,23166,74215,2011-01-18 10:01:00,1.04,C12346,United Kingdom,MEDIUM CERAMIC TOP STORAGE JAR
37955,C541433,23166,-74215,2011-01-18 10:17:00,1.04,C12346,United Kingdom,MEDIUM CERAMIC TOP STORAGE JAR
146078,554868,22197,4300,2011-05-27 10:52:00,0.72,C13135,United Kingdom,POPCORN HOLDER
314018,573008,84077,4800,2011-10-27 12:26:00,0.21,C12901,United Kingdom,WORLD WAR 2 GLIDERS ASSTD DESIGNS
377094,578841,84826,12540,2011-11-25 15:57:00,0.00,C13256,United Kingdom,ASSTD DESIGN 3D PAPER STICKERS
404447,581483,23843,80995,2011-12-09 09:15:00,2.08,C16446,United Kingdom,"PAPER CRAFT , LITTLE BIRDIE"
404448,C581484,23843,-80995,2011-12-09 09:27:00,2.08,C16446,United Kingdom,"PAPER CRAFT , LITTLE BIRDIE"


80995 ve 74215 Quantity değerine sahip ürünlerin iptal edilenler olduğu anlaşılıyor. Quantity değeri 12540 olanların da UnitPrice'ı 0 olduğundan bunların da tahmine dahil edilmemesi gerekiyor. O yüzden sildik

In [23]:
df = df.drop(df[df.Quantity==12540].index).reset_index(drop=True)
df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,C17850,United Kingdom,CREAM HANGING HEART T-LIGHT HOLDER
1,536365,71053,6,2010-12-01 08:26:00,3.39,C17850,United Kingdom,WHITE METAL LANTERN
2,536365,84406B,8,2010-12-01 08:26:00,2.75,C17850,United Kingdom,CREAM CUPID HEARTS COAT HANGER
3,536365,84029G,6,2010-12-01 08:26:00,3.39,C17850,United Kingdom,KNITTED UNION FLAG HOT WATER BOTTLE
4,536365,84029E,6,2010-12-01 08:26:00,3.39,C17850,United Kingdom,RED WOOLLY HOTTIE WHITE HEART.


## Öznitelik Mühendisliği

Dataset üzerinde yaptığımız analizlere göre çıkardığımız sonuçlar
1. İptal edilenleri boolean olarak 1 değerini olmayanları ise 0 olarak tagledik. Ve yeni bir sütun açtık
2. International sipariş olduğundan ve UK siparişleri fazlalığından UK'den olanlar 1 olmayanlar için 0 boolean değeri atadık
3. TotalPrice adında bir sütun ekleyip Quantity*UnitPrice değeri ile toplam değerine ulaştık


In [25]:
df['CancelledOrder'] = df.InvoiceNo.str.startswith('C').astype('int')

In [26]:
df.Country.value_counts()

United Kingdom          361266
Germany                   9096
France                    8168
EIRE                      7380
Spain                     2468
Netherlands               2330
Belgium                   1971
Switzerland               1844
Portugal                  1436
Australia                 1257
Norway                    1060
Italy                      783
Channel Islands            753
Finland                    653
Cyprus                     619
Sweden                     437
Austria                    387
Denmark                    375
Japan                      355
Poland                     336
USA                        291
Israel                     250
Unspecified                244
Singapore                  215
Iceland                    182
Canada                     150
Greece                     142
Malta                      123
United Arab Emirates        67
European Community          58
RSA                         57
Lebanon                     45
Lithuani

In [27]:
# 'Unspecified' ve 'European Community' olanları datasetten çıkardık
df = df.drop(df[df.Country.isin(['Unspecified','European Community'])].index).reset_index(drop=True)

In [28]:
df['InternationalOrders'] = (df.Country!='United Kingdom').astype('int')

In [29]:
df['TotalPrice'] = df.Quantity*df.UnitPrice

In [30]:
df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description,CancelledOrder,InternationalOrders,TotalPrice
0,536365,85123A,6,2010-12-01 08:26:00,2.55,C17850,United Kingdom,CREAM HANGING HEART T-LIGHT HOLDER,0,0,15.30
1,536365,71053,6,2010-12-01 08:26:00,3.39,C17850,United Kingdom,WHITE METAL LANTERN,0,0,20.34
2,536365,84406B,8,2010-12-01 08:26:00,2.75,C17850,United Kingdom,CREAM CUPID HEARTS COAT HANGER,0,0,22.00
3,536365,84029G,6,2010-12-01 08:26:00,3.39,C17850,United Kingdom,KNITTED UNION FLAG HOT WATER BOTTLE,0,0,20.34
4,536365,84029E,6,2010-12-01 08:26:00,3.39,C17850,United Kingdom,RED WOOLLY HOTTIE WHITE HEART.,0,0,20.34


In [31]:
df.to_csv('df.csv')